# Checking preprocessing actions on Fitbit data

In [20]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import preprocessing_functions
import feature_engineering_functions
from dataprep.eda import create_report

In [21]:
# load fitbit data
df = pd.read_pickle('data/daily_fitbit_df_unprocessed.pkl')

---------------------------------------------------------
Fitbit Basic Preprocessing
---------------------------------------------------------
1. Select the experiment days
2. Drop duplicates
3. Convert data types falsely described as categorical

In [22]:
df = preprocessing_functions.fitbit_basic_preprocessing(df)

----------------------------------------------
# Fitbit one-hot-encoding

bmi, age, mindfulness session, gender, activity type, badge type (is dropped)  

In [23]:
df = preprocessing_functions.fitbit_one_hot_encoding(df)

--------------------------
# Date engineering

In [24]:
df = preprocessing_functions.date_engineering(df)

In [25]:
#create_report(df).show_browser()

In [26]:
df.to_pickle('./data/fitbit_to_baseline_model.pkl')

------------------------------
# Post-preprocessing

1. Create the 3 new columns about spo2, scl_avg, heart_rate_alert before drop them
2. Drop 'spo2', 'scl_avg', and 'heart_rate_alert'
3. Replace outliers with NaNs
4. Replace NaN values with column's median for non-binary features
5. Replace NaN values with column's more frequent occurrence for binary features

In [27]:
df = preprocessing_functions.post_preprocessing(df)
df

,id,nightly_temperature,nremhr,rmssd,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,...,weekday_sin,week_sin,day_sin,month_cos,weekday_cos,week_cos,day_cos,spo2_tracking,EDA_tracking,ECG_tracking
0,621e2e8e67b776a24055b564,34.137687,57.432,89.603,14.8,78.0,0.833333,0.675,0.866667,-1.788325,...,0.0,0.309017,-0.988468,0.5,1.0,0.951057,0.151428,0,0,0
1,621e301e67b776a240608a72,34.710232,62.604,34.279,14.8,74.0,0.700000,0.700,0.833333,-1.822489,...,0.0,0.309017,-0.988468,0.5,1.0,0.951057,0.151428,0,0,0
2,621e301367b776a24057738e,NaN,62.604,34.279,14.8,75.0,0.733333,0.725,0.766667,-1.406661,...,0.0,0.309017,-0.988468,0.5,1.0,0.951057,0.151428,1,0,1
3,621e360b67b776a24039709f,NaN,68.321,31.413,15.4,69.0,0.633333,0.650,0.800000,-2.055332,...,0.0,0.309017,-0.988468,0.5,1.0,0.951057,0.151428,0,0,0
4,621e2fce67b776a240279baa,34.783747,83.170,15.941,15.2,70.0,0.733333,0.725,0.633333,-0.720949,...,0.0,0.309017,-0.988468,0.5,1.0,0.951057,0.151428,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4902,621e30e267b776a240e5bf90,NaN,62.604,34.279,14.8,75.0,0.733333,0.725,0.766667,-1.406661,...,0.0,0.809017,-0.299363,0.5,1.0,0.587785,-0.954139,0,0,0
4903,621e322367b776a240e44e9b,NaN,62.604,34.279,14.8,75.0,0.733333,0.725,0.766667,-1.406661,...,0.0,0.809017,-0.299363,0.5,1.0,0.587785,-0.954139,1,0,0
4904,621e300767b776a2404dc717,33.186399,62.604,34.279,14.8,75.0,0.733333,0.725,0.766667,-2.937423,...,0.0,0.809017,-0.299363,0.5,1.0,0.587785,-0.954139,1,0,0
4905,621e2f1b67b776a240b3d87c,34.404580,65.782,23.793,16.0,72.0,0.766667,0.725,0.666667,-2.748345,...,0.0,0.809017,-0.299363,0.5,1.0,0.587785,-0.954139,0,1,0


In [28]:
#create_report(df).show_browser()

  0%|                                                                                        | 0/11502 [00:00<…

In [29]:
df.to_pickle('./data/fitbit_post_preprocessed.pkl')